In [10]:
import os
import re
import yaml
from copy import deepcopy

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import altair as alt

In [11]:
xenium_meta = yaml.safe_load(open('../../../data/projects/xenium_grant/registration/metadata.yaml'))
xenium_meta['HT206B1-H2L1'] = deepcopy(xenium_meta['HT206B1'])
xenium_meta.pop('HT206B1')
xenium_meta

{'S18-9906': {'data': {'multiplex': {'S18-9906-U10': '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906-U10__20231006.ome.tiff',
    'S18-9906-U18': '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906-U18__20231006.ome.tiff',
    'S18-9906-U25': '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906-U25__20231006.ome.tiff',
    'S18-9906-U3': '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906-U3__20231006.ome.tiff'},
   'xenium': {'S18-9906-U17': '/diskmnt/primary/Xenium/data/20230912__220334__24002/output-XETG00122__0010784__S18-9906-B27Us1_17Q1__20230912__220421',
    'S18-9906-U2': '/diskmnt/primary/Xenium/data/20230912__220334__24002/output-XETG00122__0010784__S18-9906-B27Us1_2Q1__20230912__220421',
    'S18-9906-U24': '/diskmnt/primary/Xenium/data/20230912__220334__24002/output-XETG00122__0010787__S18-9906-B27Us1_24Q1__20230912__220421',
    'S18-9906-U9': '/diskmnt/prima

In [12]:
visium_meta = yaml.safe_load(open('../../../data/projects/subclone/registration/metadata.yaml'))
visium_meta

{'HT112C1': {'data': {'visium': {'HT112C1-U1': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U1_ST_Bn1/outs',
    'HT112C1-U2': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT112C1/H1/HT112C1-U2_ST_Bn1/outs'}},
  'ids': ['s0', 's1'],
  'order': ['HT112C1-U1', 'HT112C1-U2'],
  'registered_pixels_per_micron': 2.480402478562179,
  'scale': 0.1},
 'HT206B1': {'data': {'visium': {'HT206B1-U2': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U2Z1B1/outs',
    'HT206B1-U3': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U3Z1B1/outs',
    'HT206B1-U4': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U4Z1B1/outs',
    'HT206B1-U5': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT206B1/H1/HT206B1-S1Fc1U5Z1B1/outs'}},
  'ids': ['s0', 's1', 's2', 's3'],
  'order': ['HT206B1-U2', 'HT206B1-U3', 'HT206B1-U4', 'HT206B1-U5'],
  

In [13]:
remaining_meta = {
    'HT480B1-S1H2': {
        'order': [
            'HT480B1-S1H2-U1',
            'HT480B1-S1H2-U2',
        ],
        'data': {
            'multiplex': {
                'HT480B1-S1H2-U1': '/diskmnt/primary/CODEX/HTAN/20220917_HT480B1-S1H2_reg001/HT480B1-S1H2__20220917.ome.tiff',
                'HT480B1-S1H2-U2': '/diskmnt/primary/CODEX/HTAN/20220917_HT480B1-S1H2_reg002/HT480B1-S1H2__20220915.ome.tiff',
            }
        }
    },
    'HT308B1-S1H5A4': {
        'order': [
            'HT308B1-S1H5A4-U1',
            'HT308B1-S1H5A4-U2',
        ],
        'data': {
            'multiplex': {
                'HT308B1-S1H5A4-U1': '/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT308B1-S1H5A4_left__20230526.ome.tiff',
                'HT308B1-S1H5A4-U2': '/diskmnt/primary/CODEX/HTAN/052623_HT171B1-S1H9A1-A4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT308B1-S1H5A4_right__20230526.ome.tiff',
            }
        }
    },
    'HT206B1-H1': {
        'order': [
            'HT206B1-H1-U1',
            'HT206B1-H1-U2',
        ],
        'data': {
            'multiplex': {
                'HT206B1-H1-U1': '/diskmnt/primary/CODEX/HTAN/20220110-HT206B1-H1/HT206B1-H1__20220110.ome.tiff',
                'HT206B1-H1-U2': '/diskmnt/primary/CODEX/HTAN/20220625-HT206B1-H1/HT206B1-H1__20220625.ome.tiff',
            }
        }
    },
    'HT243B1-S1H4A4': {
        'order': [
            'HT243B1-S1H4A4-U1',
            'HT243B1-S1H4A4-U2',
            'HT243B1-S1H4A4-U3',
        ],
        'data': {
            'multiplex': {
                'HT243B1-S1H4A4-U1': '/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT243B1-S1H4A4__20230420.ome.tiff',
                'HT243B1-S1H4A4-U2': '/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT243B1-S1H4A4_left__20230512.ome.tiff',
                'HT243B1-S1H4A4-U3': '/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT243B1-S1H4A4_right__20230512.ome.tiff',
            }
        }
    },
    'HT271B1-S1H6A5': {
        'order': [
            'HT271B1-S1H6A5-U1',
            'HT271B1-S1H6A5-U2',
        ],
        'data': {
            'multiplex': {
                'HT271B1-S1H6A5-U1': '/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT271B1-S1H6A5_left__20230512.ome.tiff',
                'HT271B1-S1H6A5-U2': '/diskmnt/primary/CODEX/HTAN/051223_BRCA_HT171B1-S1H9A1-4_TOP_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4/HT271B1-S1H6A5_right__20230512.ome.tiff',
            }
        }
    },
}

In [16]:
pool = [k for d in [xenium_meta, visium_meta, remaining_meta] for k in d.keys()]
len(pool), len(set(pool))

(20, 20)

In [22]:
data = []

for d in [xenium_meta, visium_meta, remaining_meta]:
    for k, entry in d.items():
        order = entry['order']
        for dtype, mapping in entry['data'].items():
            for name, filepath in mapping.items():
                idx = order.index(name)
                data.append([k, name, dtype, idx])
df = pd.DataFrame(data=data, columns=['sample', 'section', 'dtype', 'order'])
df = df.sort_values(by=['sample', 'order'])
df
    

,sample,section,dtype,order
20,HT112C1,HT112C1-U1,visium,0
21,HT112C1,HT112C1-U2,visium,1
22,HT206B1,HT206B1-U2,visium,0
23,HT206B1,HT206B1-U3,visium,1
24,HT206B1,HT206B1-U4,visium,2
...,...,...,...,...
0,S18-9906,S18-9906-U10,multiplex,3
4,S18-9906,S18-9906-U17,xenium,4
1,S18-9906,S18-9906-U18,multiplex,5
6,S18-9906,S18-9906-U24,xenium,6


In [23]:
df.to_csv('/diskmnt/Projects/Users/estorrs/sandbox/serial_table_initial.txt', sep='\t', index=False)

In [65]:
# df = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/serial_section_table.txt', sep='\t')
df = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/Spatialomics and imaging sectioning - Sheet1.txt',
                sep='\t') 
df

,sample,section,dtype,order,microns,cancer type,confidence
0,HT112C1,HT112C1-U1,visium,0,0.0,colorectal,NaN
1,HT112C1,HT112C1-U2,visium,1,60.0,colorectal,NaN
2,HT206B1,HT206B1-U2,visium,0,0.0,breast,NaN
3,HT206B1,HT206B1-U3,visium,1,40.0,breast,NaN
4,HT206B1,HT206B1-U4,visium,2,80.0,breast,NaN
...,...,...,...,...,...,...,...
101,S18-25943-A7,S18-25943-A7-U8,xenium,3,35.0,pancreas,NaN
102,S18-25943-A7,S18-25943-A7-U9,multiplex,4,40.0,pancreas,NaN
103,S18-25943-A7,S18-25943-A7-U11,xenium,5,50.0,pancreas,NaN
104,S18-25943-A7,S18-25943-A7-U13,multiplex,6,60.0,pancreas,NaN


In [74]:
to_disease = {k:v for k, v in zip(df['sample'], df['cancer type'])}

In [78]:
from collections import Counter
counts = Counter(df['sample'].to_list())
count_df = pd.DataFrame([[k, v] for k, v in counts.most_common()])
count_df = count_df.set_index(0)
count_df.columns = ['count']
count_df['disease'] = [to_disease[x] for x in count_df.index]
count_df = count_df.sort_values(['disease', 'count'])
count_df

,count,disease
0,,
HT206B1-H1,2,breast
HT235B1,2,breast
HT243B1-H4,2,breast
HT271B1-S1H6A5,2,breast
HT308B1-S1H5A4,2,breast
HT339B1,2,breast
HT480B1-S1H2,2,breast
HT243B1-S1H4A4,3,breast
HT206B1,4,breast


In [80]:
c1 = alt.Chart(df).mark_tick(
    thickness=3,
).encode(
    x=alt.X('microns:Q', scale=alt.Scale(domain=[-10, 510])),
    y=alt.Y('sample:O', sort=count_df.index.to_list()),
    color='dtype'
)
c1

alt.Chart(...)

In [81]:
c2 = alt.Chart(df).mark_rect().encode(
#     x='x:O',
    y=alt.Y('sample:O', sort=count_df.index.to_list()),
    color=alt.Color('cancer type').scale(scheme='accent')
)
c2

alt.Chart(...)

In [82]:
alt.hconcat(c1, c2)

alt.HConcatChart(...)